# Facies classification using Machine Learning #
## LA Team Submission 5 ## 
### _[Lukas Mosser](https://at.linkedin.com/in/lukas-mosser-9948b32b/en), [Alfredo De la Fuente](https://pe.linkedin.com/in/alfredodelafuenteb)_ ####

In this approach for solving the facies classfication problem ( https://github.com/seg/2016-ml-contest. ) we will explore the following statregies:
- Features Exploration: based on [Paolo Bestagini's work](https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try02.ipynb), we will consider imputation, normalization and augmentation routines for the initial features.
- Model tuning: 

## Libraries

We will need to install the following libraries and packages.

In [31]:
# %%sh
# pip install pandas
# pip install scikit-learn
# pip install tpot

In [32]:
from __future__ import print_function
import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold , StratifiedKFold
# from classification_utilities import display_cm, display_adj_cm
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import LeavePGroupsOut
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.signal import medfilt

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
import numpy as np

## Data Preprocessing

In [33]:
#Load Data
data = pd.read_csv('../facies_vectors.csv')
# data = pd.read_csv('../ShiangYong/facies_vectors_imputedPE.csv')
# Parameters
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# data.dropna(inplace=True)
# Store features and labels
X = data[feature_names].values 
y = data['Facies'].values 

# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values
# X = np.array(pd.DataFrame(X).dropna())
# Fill 'PE' missing values with mean
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4149 entries, 0 to 4148
Data columns (total 11 columns):
Facies       4149 non-null int64
Formation    4149 non-null object
Well Name    4149 non-null object
Depth        4149 non-null float64
GR           4149 non-null float64
ILD_log10    4149 non-null float64
DeltaPHI     4149 non-null float64
PHIND        4149 non-null float64
PE           3232 non-null float64
NM_M         4149 non-null int64
RELPOS       4149 non-null float64
dtypes: float64(7), int64(2), object(2)
memory usage: 356.6+ KB


We procceed to run [Paolo Bestagini's routine](https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try02.ipynb) to include a small window of values to acount for the spatial component in the log analysis, as well as the gradient information with respect to depth. This will be our prepared training dataset.

In [35]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [36]:
X_aug, padded_rows = augment_features(X, well, depth)

In [37]:
# # Initialize model selection methods
# lpgo = LeavePGroupsOut(2)

# # Generate splits
# split_list = []
# for train, val in lpgo.split(X, y, groups=data['Well Name']):
#     hist_tr = np.histogram(y[train], bins=np.arange(len(facies_names)+1)+.5)
#     hist_val = np.histogram(y[val], bins=np.arange(len(facies_names)+1)+.5)
#     if np.all(hist_tr[0] != 0) & np.all(hist_val[0] != 0):
#         split_list.append({'train':train, 'val':val})

In [38]:
def preprocess():
    
    # Preprocess data to use in model
    X_train_aux = []
    X_test_aux = []
    y_train_aux = []
    y_test_aux = []
    
    # For each data split
    split = split_list[5]
        
    # Remove padded rows
    split_train_no_pad = np.setdiff1d(split['train'], padded_rows)

    # Select training and validation data from current split
    X_tr = X_aug[split_train_no_pad, :]
    X_v = X_aug[split['val'], :]
    y_tr = y[split_train_no_pad]
    y_v = y[split['val']]

    # Select well labels for validation data
    well_v = well[split['val']]

    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
        
    X_train_aux.append( X_tr )
    X_test_aux.append( X_v )
    y_train_aux.append( y_tr )
    y_test_aux.append (  y_v )
    
    X_train = np.concatenate( X_train_aux )
    X_test = np.concatenate ( X_test_aux )
    y_train = np.concatenate ( y_train_aux )
    y_test = np.concatenate ( y_test_aux )
    
    return X_train , X_test , y_train , y_test 

## Data Analysis

In this section we will run a Cross Validation routine 

In [39]:
# from tpot import TPOTClassifier
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = preprocess()

# tpot = TPOTClassifier(generations=5, population_size=20, 
#                       verbosity=2,max_eval_time_mins=20,
#                       max_time_mins=100,scoring='f1_micro',
#                       random_state = 17)
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('FinalPipeline.py')

In [40]:
from sklearn.ensemble import  RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier

In [41]:
# Train and test a classifier

# Pass in the classifier so we can iterate over many seed later.
def train_and_test(X_tr, y_tr, X_v, well_v, clf):
    
    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
    
    clf.fit(X_tr, y_tr)
    
    # Test classifier
    y_v_hat = clf.predict(X_v)
    
    # Clean isolated facies for each well
    for w in np.unique(well_v):
        y_v_hat[well_v==w] = medfilt(y_v_hat[well_v==w], kernel_size=5)
    
    return y_v_hat

In [42]:
# Train and test a classifier

# Pass in the classifier so we can iterate over many seed later.
def train_and_test_non_validation(X_tr, y_tr, X_v, well_v, clf):
    
    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
    
    clf.fit(X_tr, y_tr)
    
    # Test classifier
    y_v_hat = clf.predict(X_v)
    
    # Clean isolated facies for each well
#     for w in np.unique(well_v):
#         y_v_hat[well_v==w] = medfilt(y_v_hat[well_v==w], kernel_size=5)
    
    return y_v_hat

## Prediction

In [43]:
#Load testing data
test_data = pd.read_csv('../validation_data_nofacies.csv')

    # Train classifier
    #clf = make_pipeline(make_union(VotingClassifier([("est", ExtraTreesClassifier(criterion="gini", max_features=1.0, n_estimators=500))]), FunctionTransformer(lambda X: X)), XGBClassifier(learning_rate=0.73, max_depth=10, min_child_weight=10, n_estimators=500, subsample=0.27))
    #clf =  make_pipeline( KNeighborsClassifier(n_neighbors=5, weights="distance") ) 
    #clf = make_pipeline(MaxAbsScaler(),make_union(VotingClassifier([("est", RandomForestClassifier(n_estimators=500))]), FunctionTransformer(lambda X: X)),ExtraTreesClassifier(criterion="entropy", max_features=0.0001, n_estimators=500))
    # * clf = make_pipeline( make_union(VotingClassifier([("est", BernoulliNB(alpha=60.0, binarize=0.26, fit_prior=True))]), FunctionTransformer(lambda X: X)),RandomForestClassifier(n_estimators=500))

# # Prepare training data
# X_tr = X
# y_tr = y

# # Augment features
# X_tr, padded_rows = augment_features(X_tr, well, depth)

# # Removed padded rows
# X_tr = np.delete(X_tr, padded_rows, axis=0)
# y_tr = np.delete(y_tr, padded_rows, axis=0) 

# Prepare test data
well_ts = test_data['Well Name'].values
depth_ts = test_data['Depth'].values
X_ts = test_data[feature_names].values


    
y_pred = []
print('.' * 100)
for seed in range(3):
    np.random.seed(seed)
    
    # Hold out two wells
    ind_shk = np.array(data[data['Well Name']=='SHANKLE'].index)
    ind_chr = np.array(data[data['Well Name']=='CHURCHMAN BIBLE'].index)
    ind_ho_set = np.append(ind_shk,ind_chr)

    # Make training data.
    X_train, padded_rows = augment_features(X, well, depth)
    y_train = y
    X_test_nv = np.take(X_train, ind_ho_set, axis=0) 
    y_test_nv = np.take(y_train, ind_ho_set, axis=0) 
    X_train_nv = np.delete(X_train, ind_ho_set, axis=0)
    y_train_nv = np.delete(y_train, ind_ho_set, axis=0)
    
    X_train_nv = np.delete(X_train_nv, padded_rows, axis=0)
    y_train_nv = np.delete(y_train_nv, padded_rows, axis=0) 
    
    X_test_nv = np.delete(X_test_nv, padded_rows, axis=0)
    y_test_nv = np.delete(y_test_nv, padded_rows, axis=0)

    # Train classifier  
    clf = make_pipeline(XGBClassifier(learning_rate=0.12,
                                      max_depth=3,
                                      min_child_weight=10,
                                      n_estimators=150,
                                      seed=seed,
                                      colsample_bytree=0.9))

    
    
    # Make blind data.
#     X_test, _ = augment_features(X_ts, well_ts, depth_ts)
    

    # Train and test.
#     y_ts_hat = train_and_test(X_train, y_train, X_test, well_ts, clf)
    
    # Collect result.
#     y_pred.append(y_ts_hat)
#     print('|', end='')
    
# np.save('LA_Team_100_realizations.npy', y_pred)

....................................................................................................


In [44]:
set(padded_rows) & set(ind_ho_set)

{937, 3745, 3784}

In [45]:
padded_rows.shape

(18,)

In [46]:
X_test_nv.shape

(850, 28)

In [47]:
ind_ho_set.shape

(853,)

In [48]:
y_train.shape

(4149,)

In [49]:
# X_train_nv, X_test_nv, y_train_nv, y_test_nv = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
# np.delete(X_train, ind_ho_set, axis=0)



In [50]:
y_pred = train_and_test_non_validation(X_train_nv, y_train_nv, X_test_nv, well_ts, clf)

In [51]:
correct = 0
tot = 0
for i, entry in enumerate(confusion_matrix(y_pred,y_test_nv)):
    for j, e in enumerate(entry):
        if i == j:
            correct += e
        tot += e
print(correct/tot)

0.571764705882


In [52]:
f1_score(y_pred,y_test_nv, average='micro')

0.57176470588235295

In [53]:
accuracy_score(y_pred,y_test_nv)

0.57176470588235295

### attempt at using LSTM for including influence of previous features

In [54]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, (2*N_neig+1), N_feat))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
#         print(this_row.shape)
        this_row.shape = ((2*N_neig+1), this_row.size // (2*N_neig+1))
#         print(this_row)
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature augmentation function
def augment_features(X, well, depth, N_neig=12):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], (N_neig*2+1), X.shape[1]))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
#         X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = X_aug_win
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [55]:
# model.save('LSTM_acc_74.h5', overwrite=True)
# y_pred = model.predict(X_test_nv_LSTM, batch_size=20, verbose=0)
# y_test_LSTM_ct.shape
# predicted_classes = np.argmax(y_pred, axis=1)
# class_labels = np.argmax(y_test_LSTM_ct, axis=1)
# f1_score(predicted_classes,class_labels, average='micro') # micro is the same as accuracy in this problem

### Selection to apply LSTM w held out well 

In [56]:
#Load Data
data = pd.read_csv('../facies_vectors.csv')
# data = pd.read_csv('../ShiangYong/facies_vectors_imputedPE.csv')
# Parameters
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# data.dropna(inplace=True)
# Store features and labels
X = data[feature_names].values 
y = data['Facies'].values 

# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values
# X = np.array(pd.DataFrame(X).dropna())
# Fill 'PE' missing values with mean
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)

# NEEDS TO BE CHANGED, SCALING SHOULD NOT BE DETERMINED FROM TESTING AND TRAINING SET, ONLY TRAINING
# scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X)
# X = scaler.transform(X)

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
# X_test_nv_LSTM = scaler.transform(X)

Get indicies of wells that will make up hold out set

In [62]:
X_train, padded_rows = augment_features(X, well, depth)
y_train = y
X_test_LSTM = np.take(X_train, ind_ho_set, axis=0) 
y_test_LSTM = np.take(y_train, ind_ho_set, axis=0) 
X_train_LSTM = np.delete(X_train, ind_ho_set, axis=0)
y_train_LSTM = np.delete(y_train, ind_ho_set, axis=0)

In [63]:
X_train.shape

(4149, 25, 7)

In [64]:
X_test_LSTM = X_test_LSTM[0:840]
y_test_LSTM = y_test_LSTM[0:840]
X_train_LSTM = X_train_LSTM[0:3280]
y_train_LSTM = y_train_LSTM[0:3280]

In [65]:
# X_train_nv_LSTM, X_test_nv_LSTM, y_train_nv_LSTM, y_test_nv_LSTM = train_test_split(X_train[0:4000], y_train[0:4000], test_size=0.3, random_state=42)
y_train_LSTM = y_train_LSTM - 1
y_test_LSTM = y_test_LSTM - 1
y_train_LSTM_ct = to_categorical(np.array(y_train_LSTM), num_classes=None)
y_test_LSTM_ct = to_categorical(np.array(y_test_LSTM), num_classes=None)

In [66]:
'''Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
# Notes
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''




data_dim = 7 # Features
timesteps = 25 # 25 is best so far observed
num_classes = 9
batch_size = 20 # 20 is best so far observed

# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.
model = Sequential()


model.add(LSTM(40, stateful=True, return_sequences=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))

# model.add(LSTM(40, return_sequences=True, stateful=True))

model.add(LSTM(40, stateful=True))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

model.fit(X_train_LSTM, y_train_LSTM_ct,
          batch_size=batch_size, epochs=30, shuffle=False,
          validation_data=(X_test_LSTM, y_test_LSTM_ct))

Train on 3280 samples, validate on 840 samples
Epoch 1/30
3280/3280 [==============================] - 6s 2ms/step - loss: 1.6064 - acc: 0.3774 - val_loss: 1.5734 - val_acc: 0.4536
Epoch 2/30
3280/3280 [==============================] - 5s 2ms/step - loss: 1.3494 - acc: 0.4445 - val_loss: 1.5114 - val_acc: 0.4690
Epoch 3/30
3280/3280 [==============================] - 5s 2ms/step - loss: 1.2369 - acc: 0.4814 - val_loss: 1.4920 - val_acc: 0.4560
Epoch 4/30
3280/3280 [==============================] - 6s 2ms/step - loss: 1.1735 - acc: 0.5046 - val_loss: 1.4417 - val_acc: 0.4321
Epoch 5/30
3280/3280 [==============================] - 7s 2ms/step - loss: 1.1223 - acc: 0.5274 - val_loss: 1.4163 - val_acc: 0.4060
Epoch 6/30
3280/3280 [==============================] - 4s 1ms/step - loss: 1.0454 - acc: 0.5573 - val_loss: 1.4403 - val_acc: 0.4214
Epoch 7/30
3280/3280 [==============================] - 4s 1ms/step - loss: 1.0187 - acc: 0.5826 - val_loss: 1.4371 - val_acc: 0.4250
Epoch 8/30
3280